# NOTEBOOK 0: MIGROS DATA DISCOVERY

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib notebook
sns.set(style="ticks", font_scale=1.2, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (18, 7)

Import the 15 minutes data sample from Migros (2 stations Aadorf and Zuzwill) (after I cleaned the data and took out manually useless headers)

In [ ]:
migros_data = pd.read_csv("data/AADO_and_ZUZW_ConsumptionRawValues_CLEAN.csv",delimiter=",",header=0,encoding="ISO-8859-1")
migros_data

In [ ]:
Station_locations = migros_data.columns[1:]
Station_locations

Let's keep only the total electricity, heat consumption, warm heating, for both locations. (The other columns are sub-consumptions for sub-markets or rooms within the migros location).
So it is 3 * 2 stations, * 12 months * 3 years = 216 monthly images. 

In [ ]:
3*2*12*3

In [ ]:
Station_locations_selected0 = ['Stromverbrauch Gesamt','Wrmeverbrauch Gesamt','Wasserverbrauch Gesamt','Stromverbrauch Gesamt.1','Wrmeverbrauch Gesamt.1','Wasserverbrauch Gesamt.1']

Select the right columns in the data. 

In [ ]:
migros_data_selection = migros_data[Station_locations_selected0]
migros_data_selection.index = migros_data["Description"]
# rename column names
migros_data_selection = migros_data_selection.rename(
    columns={"Stromverbrauch Gesamt": "Aadorf_tot_elec", 
                   "Wrmeverbrauch Gesamt": "Aadorf_tot_heating",
                  "Wasserverbrauch Gesamt": "Aadorf_tot_water",
                  "Stromverbrauch Gesamt.1": "Zuzwil_tot_elec", 
                   "Wrmeverbrauch Gesamt.1": "Zuzwil_tot_heating",
                  "Wasserverbrauch Gesamt.1": "Zuzwil_tot_water"})

Station_locations_selected = migros_data_selection.columns

In [ ]:
migros_data_selection

Look at times series, for the selected columns 


In [ ]:
station_list = []
# the first 48 colums are apriori the locations with the demand; then the temperature or others

for station_no in range(len(Station_locations_selected)):
    # plot just the demand values (first 5 rows are other things)
    station = migros_data_selection.iloc[5:,station_no].fillna(-1)
    # set up the time stamp as the index
    station.index = pd.DatetimeIndex(migros_data_selection.index[5:])
    
    ## treat the last station because for some reason the values are not numerical but strings?? TRASNFORM THE STRINGS INTO NUMBERS
    #if station_no == 47:
    #    station = pd.Series(np.array([ float(el) for el in station]))
    #    station.index = pd.DatetimeIndex(data.iloc[5:,0])
    
    station_list.append(station)

In [ ]:
# transform in float values everywhere; there are some values recorded as strings...
station_list2 = []
for i in range(len(station_list)):
    
    # float each number, while keeping the series format
    station2 = pd.Series(np.array([float(el) for el in station_list[i]]))
    # keeping the datetimes as indexes
    station2.index = station_list[i].index
    station_list2.append(station2)

plot times series for the two total electricity stations 

In [ ]:
sns.set(style="ticks", font_scale=1.2, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (15, 12)

fig = plt.figure()

plt.subplot(2,1,1)
station_list2[0].plot()
plt.title(Station_locations_selected[0])

plt.subplot(2,1,2)
station_list2[3].plot()
plt.title(Station_locations_selected[3])#fig.tight_layout()

plt.show()

#### We are supposed to give flags at week resolution. So let's see hows things are doing at week scale.

In [ ]:
def TimeFormat(year, month, day):
    temp = "".join([str(year),"-",str(month),"-",str(day)])
    return(temp)

Plot for one station, one year, one month, the 4 weeks demand.

In [ ]:
sns.set(style="ticks", font_scale=1.2, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (15, 10)


# station in Aadorf
station_no = 0
year_no,month_no = 2017,1

# plot the 4 weeks in January (excluding the 3 last days, that allows to plot all the months in the same manner.)
fig=plt.figure()
for i in range(4):
    plt.subplot(4,1,i+1)
    station_week = station_list2[station_no][TimeFormat(year_no, month_no, 1 + 7*i):TimeFormat(year_no, month_no, 7*(i+1))]
    # plot each week with the original 15 minutes resolution.
    #station_week.plot()
    ## resample by hour instead of 15 minutes
    station_week.resample('h').mean().plot()
    plt.xlabel("Electricity demand")
    if i==1:
        plt.ylabel("Hours of the day")
    #plt.axis([1 + 7*i, 7*(i+1),0,24])
#fig.tight_layout()
plt.show()        

In [ ]:
# example week
week = station_list2[station_no][TimeFormat(year_no, month_no, 1 + 7*i):TimeFormat(year_no, month_no, 7*(i+1))]
# resample for each hour instead of 15 minutes
week_hours = week.resample('h').mean()
# just fold the data every 24 hours to make the (hours,week days) matrix 
week_matrix = np.array([week_hours[ 24*j:24*(j+1)] for j in range(7)]).T
week_matrix

#### Entire year plot, from march (to match the study); for one station

In [ ]:
# PLOT
def PlotMonthDemand(station,year,month, vmin=None,vmax=None,save=False):
    mon = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

    fig = plt.figure()
    
    # select 2018 from march to march 2019, like in the survey
    month_series = station_list2[station][TimeFormat(year, month, 1 ):TimeFormat(year, month, 28)]
    # resample for each hour instead of 15 minutes
    #year_series_hours = year_series.resample('h').mean()
    # just fold the data every 24 hours to make the (hours,week days) matrix 
    month_series_matrix = np.array([month_series[ 96*j:96*(j+1)][::-1] for j in range(int(len(month_series)/96))]).T
    
    # plot
    plt.imshow(month_series_matrix, aspect="auto", cmap="gray",vmin=vmin,vmax=vmax,extent=[0,int(len(month_series)/96),0,96])    
    plt.colorbar()
    plt.title("".join(["Station : ",Station_locations_selected[station], ", Year: ", str(year) ,", Month: ", mon[month-1] ] ) )

    plt.xlabel("Days of the month")
    plt.ylabel("15 minutes groups in the day")
    fig.tight_layout()
    
    
    # to save if you want
    if save == True:
        name = ''.join(["Migros_energy_images/MonthImage_",Station_locations_selected[station] ,"_",str(year),"_",str(month)  ])
        fig.savefig(name,dpi=300,bbox_inches="tight")

Year demand, from january to december this time (this is not Swisscom data)

In [ ]:
year="2018"
month = "12"
''.join([Station_locations_selected[0] ,"_",year,"_",month  ])

In [ ]:

def PlotYearDemand(station, year,vmin=None,vmax=None):
    #fig = plt.figure()
    fig, ax = plt.subplots(1,1)
    #  
    year_series = station_list2[station][TimeFormat(year, 1, 1 ):TimeFormat(year, 12, 31)]
    # resample for each hour instead of 15 minutes
    #year_series_hours = year_series.resample('h').mean()
    # just fold the data every 24 hours to make the (hours,week days) matrix 
    year_series_matrix = np.array([year_series[ 96*j:96*(j+1)][::-1] for j in range(int(len(year_series)/96))]).T
    
    # plot
    plt.imshow(year_series_matrix, aspect="auto", cmap="jet",vmin=vmin,vmax=vmax,extent=[0,int(len(year_series)/96),0,96])    
    plt.colorbar()
    plt.title("".join(["Station : ",Station_locations_selected[station]]) )
    plt.xlabel("Days of the year")
    plt.ylabel("15 minutes groups in the day")
    
    #x_label_list = list(year_series.index)
    #ax.set_xticks([0,int(len(year_series)/96),96,0])
    #ax.set_xticklabels(x_label_list)
    
    fig.tight_layout()
    plt.show()


Plot!

In [ ]:
sns.set(style="ticks", font_scale=1.2, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (28, 7)

One station, different years

In [ ]:
station_no = 0
PlotYearDemand(station_no,2016)

In [ ]:
station_no = 0
PlotYearDemand(station_no,2017)

In [ ]:
station_no = 0
PlotYearDemand(station_no,2018)

Same station, at month level.

In [ ]:
station_no = 0
PlotMonthDemand(station_no,2016,7, vmin=0)


In [ ]:
# bonaduz
station_no = 0
PlotMonthDemand(station_no,2017,1,save=True)

In [ ]:
# bonaduz
station_no = 0
PlotMonthDemand(station_no,2016,11)

##### Let's plot all month images, for all 2017 and 2018 (issues and missing months in 2016 and 2019), for the 6 stations (i.e. 2 locations for the 3 energy types)

In [ ]:
for station in range(len(Station_locations_selected)):
    for year in (el for el in [2017,2018]):
        for month in range(1,13):
            PlotMonthDemand(station,year,month,save=True)
            print("next month...")   
        print("next year... pfouahh")
    print("Current energy type done...")

### Circle representation (not explored anymore)

For the circle representation.

In [ ]:
def MakeMonthArray(station,year,month):
    mon = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

    # select 2018 from march to march 2019, like in the survey
    month_series = station_list2[station_no][TimeFormat(year, month, 1 ):TimeFormat(year, month, 28)]
    # just fold the data every 24 hours to make the (hours,week days) matrix 
    month_series_matrix = np.array([month_series[ 96*j:96*(j+1)][::-1] for j in range(int(len(month_series)/96))]).T
    
    return(month_series_matrix)

In [ ]:
def ComputeCMandRadius(station,year,month, radius="Sum",background=0):
    # compute the monthly array of consumption
    MonthlyArray = MakeMonthArray(station,year,month)

    morning_CMs, afternoon_CMs = [], []
    morning_means, afternoon_means = [], []
    morning_sums, afternoon_sums = [], []

    # loops over the days
    for i in range(28):
    
        #-------------------- compute CM = location of circle centers --------------------
        # lists of weights 
        mornings_day, afternoons_day = MonthlyArray[48:][:,i], MonthlyArray[:48][:,i]
        # lists of x (positions)
        mornings_day_x,afternons_day_x = [i for i in range(48)] , [i for i in range(48,96)] 

        # center of mass = sum(x*weights)/sum(weights)
        mornings_day_CM = np.sum(mornings_day*mornings_day_x)/np.sum(mornings_day)
        afternoons_day_CM = np.sum(afternoons_day*afternons_day_x)/np.sum(afternoons_day)

        morning_CMs.append(mornings_day_CM)
        afternoon_CMs.append(afternoons_day_CM)
        
        #-------------------- compute radii of circles --------------------
        if not (-1 in mornings_day) and not (-1 in afternoons_day):
            # with the mean energy
            #mornings_enMean, afternoons_enMean = np.mean(mornings_day) , np.mean(afternoons_day)
            
            # with the cumulative sum energy
            #mornings_enSum, afternoons_enSum = np.sum(np.array(mornings_day)-background) , np.sum(np.array(afternoons_day)-background)
    
            # scaled series
            mornings_scaled, afternoons_scaled = (mornings_day-min(mornings_day))/float(max(mornings_day)-min(mornings_day)) , (afternoons_day-min(afternoons_day))/float(max(afternoons_day)-min(afternoons_day))
            mornings_scaled_enSum, afternoons_scaled_enSum = np.sum(np.array(mornings_scaled)) , np.sum(np.array(afternoons_scaled))       
            
            #if i==27:
            #   print(afternoons_day)
            #   print(afternoons_scaled)
        else:
            #mornings_enMean, afternoons_enMean = 0,0
            mornings_scaled_enSum, afternoons_scaled_enSum = 0,0        
            
        #morning_means.append(mornings_enMean)
        #afternoon_means.append(afternoons_enMean)
        morning_sums.append(mornings_scaled_enSum)
        afternoon_sums.append(afternoons_scaled_enSum)
                
        # we divide by 10/1000 the mean/sum radius so that they are in an order of magnitude that makes sense
    if radius == "Sum":
        return((morning_CMs, afternoon_CMs,np.array(morning_sums)/10, np.array(afternoon_sums)/10))
    if radius == "Mean":
        return((morning_CMs, afternoon_CMs,np.array(morning_means)/10, np.array(afternoon_means)/10))

In [ ]:
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

Plot circles for one example

In [ ]:
sns.set(style="ticks", font_scale=1.2, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (10, 5)

# example station, year, and month; same station 0 and year 2017 as before; with the right "background"
ex = ComputeCMandRadius(0,2017,1,"Sum",0)

#### ------ Plot resulting energy circles -----------


# x locations of the circles, being each day of the month
x_morning , x_afternoon = 2*np.array([i for i in range(28)]), 2*np.array([i for i in range(28)])
# y locations of the circles, being the center of mass we computed before; 
# NOTE: to have nice circles, we need to normalize the 15 minutes scale
y_morning , y_afternoon = np.array(ex[0])*0.291, np.array(ex[1])*0.291
# radii of the circles, as the mean of the energy
rad_morning, rad_afternoon = ex[2], ex[3]

patches_morning, patches_afternoon = [],[]

# morning circles
for x1, y1, r1 in zip(x_morning, y_morning, rad_morning):
    circle = Circle((x1, y1), r1)
    patches_morning.append(circle)

# afternoon circles
for x2, y2, r2 in zip(x_afternoon, y_afternoon, rad_afternoon):
    circle = Circle(( x2, y2), r2)
    patches_afternoon.append(circle)
    
# -------- -------- PLOT --------------------------------------------

fig, ax = plt.subplots()
p1 = PatchCollection(patches_morning, alpha=0.2)
p2 = PatchCollection(patches_afternoon, alpha=0.2)
ax.add_collection(p1)
ax.add_collection(p2)
ax.set_xlim([0,56])
ax.set_ylim([0,28])
#plt.axis('off')
plt.savefig("figure.png",dpi=300, bbox_inches="tight")
plt.show()

## Detection of local anomalies

In [ ]:
def TimeFormat(year, month, day):
    temp = "".join([str(year),"-",str(month),"-",str(day)])
    return(temp)

sns.set(style="ticks", font_scale=1, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (13, 2)

Make functions to plot monthly and yearly time series for any of the two stations. 

In [ ]:
def PlotMonthlyTimesSeries(year_no,month_no,station="Aadorf",resolution="15min"):

    fig=plt.figure()
    
    if station == "Aadorf":
        station_no=0
    elif station == "Zuzwil":
        station_no=3
    
    station_month = station_list2[station_no][TimeFormat(year_no, month_no, 1):TimeFormat(year_no, month_no, 28 )]
    # plot each week with the original 15 minutes resolution.
    if resolution == "15min":
        station_month.plot()
    if resolution == "hourly":
        station_month.resample('h').mean().plot()
    if resolution == "daily":
        station_month.resample('D').mean().plot()
    ## resample by hour instead of 15 minutes
    #station_week.resample('h').mean().plot()
    plt.xlabel("Days of the month")
    plt.ylabel("Electricity demand")
    #plt.axis([1 + 7*i, 7*(i+1),0,24])
    #fig.tight_layout()
    plt.show()        

In [ ]:
def PlotYearlyTimesSeries(year_no,station="Aadorf",resolution="15min"):

    fig=plt.figure()
    
    if station == "Aadorf":
        station_no=0
    elif station == "Zuzwil":
        station_no=3
    
    station_month = station_list2[station_no][TimeFormat(year_no, 1, 1):TimeFormat(year_no, 12, 28 )]
    # plot each week with the original 15 minutes resolution.
    if resolution == "15min":
        station_month.plot()
    if resolution == "hourly":
        station_month.resample('h').mean().plot()
    if resolution == "daily":
        station_month.resample('D').mean().plot()
    ## resample by hour instead of 15 minutes
    #station_week.resample('h').mean().plot()
    plt.xlabel("Days of the year")
    plt.ylabel("Electricity demand")
    #plt.axis([1 + 7*i, 7*(i+1),0,24])
    #fig.tight_layout()
    plt.show() 

#### Example of no issue...

In [ ]:
PlotMonthlyTimesSeries(2018,1, "Aadorf","15min")


#### Examples of issues...

In [ ]:
PlotMonthlyTimesSeries(2018,4, "Aadorf","15min")

###### These ones are good because the anomaly is at the end!

In [ ]:
PlotMonthlyTimesSeries(2018,12, "Aadorf","hourly")

In [ ]:
PlotMonthlyTimesSeries(2018,12, "Zuzwil","hourly")

## Try to train a model, and forecast + detect anomalies

Select one series with an anomaly: Aadorf, 2018, December, hourly aggregated. 

In [ ]:
Aadorf_2018_dec_hourly = station_list2[0][TimeFormat(2018, 12, 1):TimeFormat(2018, 12, 28 )].resample('h').mean()
Aadorf_2018_dec_hourly = pd.DataFrame(Aadorf_2018_dec_hourly,columns=["Demand"])
Aadorf_2018_dec_hourly

Features: time lags! In this case we have daily period but also a monthly period! SO we need to capture that with a lot of lags...

In [ ]:
# Creating a copy of the initial datagrame to make various transformations 
data = pd.DataFrame(Aadorf_2018_dec_hourly.Demand.copy())
data.columns = ["y"]

# Adding the lag of the target variable from 6 steps back up to 7 days * 24= 168 lags, at least 
for i in range(6, 169):
    data["lag_{}".format(i)] = data.y.shift(i)

Features 2: hours, days of weeks, week ends etc.

In [ ]:
data.index = pd.to_datetime(data.index)
data["hour"] = data.index.hour
data["weekday"] = data.index.weekday
data['is_weekend'] = data.weekday.isin([5,6])*1

In [ ]:
data.shape

In [ ]:
# take a look at the new dataframe 
data.tail(7)

In [ ]:
# form labelled data 
ylabel = data.dropna().y
Xlabel = data.dropna().drop(['y'], axis=1)

Let's try an RF just for our love for them....

In [ ]:
from TrainingMethods188 import CustomRF

In [ ]:
rf,rmse,RRMSE, oob = CustomRF(Xlabel,ylabel).Fast_TrainAndTest(300,100,20)

In [ ]:
rf,rmse,RRMSE, oob = CustomRF(Xlabel,ylabel).Fast_TrainAndTest(300,1,3)

In [ ]:
# form labelled data 
#ylabel = data.dropna().y
#Xlabel = data.dropna().drop(['y'], axis=1)

#rf,rmse,RRMSE, oob = CustomRF(Xlabel,ylabel).Fast_TrainAndTest(300,1,3)

# reform ytest and Xtest just to see how close the predictions are. 
ytest = ylabel[int(0.25*len(ylabel)):]
Xtest = Xlabel[int(0.25*len(ylabel)):]

# predict on the test set 
ypred0 = rf.predict(Xtest)
ypred = pd.Series(ypred0,index = ytest.index)

# plot 
plt.figure()
plt.plot(ytest,"g",label="Observed")
plt.plot(ypred,"r",label="RF predicted")
plt.title("nRMSE {0:.2f}%".format(100*RRMSE))
plt.legend(loc="upper left")
plt.show()

In [ ]:
# reform ytest and Xtest just to see how close the predictions are. 
ytest = ylabel[int(0.5*len(ylabel)):]
Xtest = Xlabel[int(0.5*len(ylabel)):]

# predict on the test set 
ypred0 = rf.predict(Xtest)
ypred = pd.Series(ypred0,index = ytest.index)

# plot 
plt.figure()
plt.plot(ytest,"g",label="Observed")
plt.plot(ypred,"r",label="RF predicted")
plt.title("nRMSE {0:.2f}%".format(100*RRMSE))
plt.legend(loc="upper left")
plt.show()

Well, we kind of detect anomalies already! It captures well the week seansonality. 

The prediction of sunday is well different and the next tuesday and wednesday are indeed well seen as strange as well... 